In [105]:
import numpy as np
import pandas as pd
import statistics as stts

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import VotingClassifier

<img src="concept_stacking.png" alt="Drawing" style="width: 800px;"/>

### 1.Import data

In [106]:
data = pd.read_csv('../data/data_cleaned.csv')

X, y = data.drop('Survived', axis=1), data.pop('Survived')

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

### 2. Creade models

In [107]:
DT = DecisionTreeClassifier()
KNN = KNeighborsClassifier()
LR = LogisticRegression(solver='lbfgs', max_iter=1000)

### 3. Evaluate each model

In [108]:
DT.fit(X_train, y_train)
KNN.fit(X_train, y_train)
LR.fit(X_train, y_train)


DT.score(X_test, y_test), KNN.score(X_test, y_test) , LR.score(X_test, y_test)

(0.8044692737430168, 0.6983240223463687, 0.8547486033519553)

#### 3.1 Second approach: using out-of-folds predictions

In [109]:
from sklearn.model_selection import cross_val_predict


def model_predictions_kfold(model, kfolds, X_train, y_train, X_test):
	
	y_train_pred = cross_val_predict(model, X_train, y_train, cv=kfolds)

	y_test_pred = model.predict(X_test)
		
	return y_train_pred, y_test_pred

#### a) Train models

In [110]:
M1_train, M1_test = model_predictions_kfold(DT, 10, X_train, y_train, X_test)
M2_train, M2_test = model_predictions_kfold(KNN, 10, X_train, y_train, X_test)
M3_train, M3_test = model_predictions_kfold(LR, 10, X_train, y_train, X_test)

#### b) Create the base models predictions dataset

In [111]:
X_train_meta = pd.DataFrame({'m1':M1_train, 'm2':M2_train, 'm3':M3_train})
X_test_meta = pd.DataFrame({'m1':M1_test, 'm2':M2_test, 'm3':M3_test})


#### c) Make predictions

In [112]:
# Stacker Model
meta_model = LogisticRegression()
meta_model.fit(X_train_meta, y_train)
meta_model.score(X_test_meta,y_test)

0.8547486033519553